In [1]:
import perceval as pcvl
import numpy as np
import sympy as sp
from math import comb
from scipy.optimize import minimize
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
import tqdm as tqdm

In [324]:
n = 3 # number of photons
m = 5 # number of modes 

bs = pcvl.BasicState([1]*n + [0]*(m-n))


There is 1 photon in mode 0
There is 1 photon in mode 1
There is 1 photon in mode 2
There is 0 photon in mode 3
There is 0 photon in mode 4


10

In [54]:



def analyzer_to_statevector(analyzer):
    r = 0*pcvl.StateVector(m*[0])
    for state,prob in zip(analyzer.output_states_list,analyzer.distribution[0]):
         r += np.sqrt(prob)*pcvl.StateVector(state)
    return r

def sim_to_statevector(sim,input_state):
    r = 0*pcvl.StateVector(m*[0])
    for prob,state in zip(sim.all_prob(input_state),sim.allstate_iterator(input_state)):
        r += np.sqrt(prob)*pcvl.StateVector(state)
    return r


In [326]:
#c = pcvl.Unitary(pcvl.Matrix.random_unitary(m, parameters[2*m**2:]))\
#// (0, pcvl.PS(px))
#//  pcvl.Unitary(pcvl.Matrix.random_unitary(m, parameters[:2*m**2]))

def generate_all_probs(x,params,circ1,circ2,angle_param):

    t1 = time.time()
    #px.reset()
    angle_param.set_value(x * np.pi)
    #c = pcvl.Unitary(pcvl.Matrix.random_unitary(m, parameters[2*m**2:]))\
    #// (0, pcvl.PS(px))\
    #//  pcvl.Unitary(pcvl.Matrix.random_unitary(m, parameters[:2*m**2]))
    simulator_backend = pcvl.BackendFactory().get_backend("SLOS")
    s1 = simulator_backend(pcvl.Matrix.random_unitary(m))
    s1.compile(bs)
    s1.U = circ1.compute_unitary(use_symbolic=False)
    t2 = time.time()
    #print ('first sim took', t2 - t1, 'seconds')
    #p=pcvl.Processor("SLOS",circ1)
   
    r = sim_to_statevector(s1,bs)
    t25 = time.time()
    #print ('sim2state took', t25 - t2, 'seconds')
    #p.with_input(bs)
    #analyzer= pcvl.algorithm.Analyzer(p,[bs],'*')
    #return analyzer.distribution[0]
    #n1 = time.time()
    #r = analyzer_to_statevector(analyzer)
    #print('time is ',n1)
    map_measure_sv = r.measure(0)
    t3 = time.time()
    #print ('measure took', t3 - t25, 'seconds')
    sv_new = map_measure_sv[pcvl.BasicState('|1>')][1]
    #print(sv_new)
    #r1 = 0*pcvl.StateVector((m-1)*[0])
    #for i,j in r.items():

    #    if i[0] == 1:
    #        print(i,j)
    #        r1+= np.sqrt(j)*pcvl.StateVector(i[1:])
    t4 = time.time()
    #print ('manual measure took', t4 - t3, 'seconds')
    #r1.normalize()
    #print(r1)
    #c2 = pcvl.Unitary(pcvl.Matrix.random_unitary(m-1, parameters1))\
    # // pcvl.Unitary(pcvl.Matrix.random_unitary(m-1, params))

 
    sv_new.normalize() # this is needed due a Perceval bug
    #print(sv_new)
    t45 = time.time()
    #print ('norm took', t45 - t4, 'seconds')
    all_probs = fock_dim2*[0]
    t5 = time.time()
    s2 = simulator_backend(pcvl.Matrix.random_unitary(m-1))
        #s2.compile(bs)
    s2.U = circ2.compute_unitary(use_symbolic=False)
    for state1,prob_amp in sv_new.items(): 
        prob = prob_amp**2
        #simulator_backend = pcvl.BackendFactory().get_backend("SLOS")
    #    s1 = simulator_backend(circ2)
    #    s1.compile(state1)
    #simulator_backend = pcvl.BackendFactory().get_backend("SLOS")
        #s2 = simulator_backend(pcvl.Matrix.random_unitary(m-1))
        #s2.compile(bs)
        #s2.U = circ2.compute_unitary(use_symbolic=False)
    #print(np.sort(s1.all_prob(bs)))
    #pcvl.pdisplay(c2)
    #print(np.sort(analyzer.distribution[0]))
    #print(prob*s1.all_prob(state1))
        all_probs+=prob*s2.all_prob(state1)
    t6 = time.time()   
    
    #print ('sim2 loop took', t6 - t5, 'seconds')
    #print(sum(all_probs))
    return all_probs
    #return analyzer.distribution[0]
    #return s1.all_prob(bs)
    
    


In [ ]:
p=pcvl.Processor("SLOS",u)
sv = pcvl.StateVector(bs)
p.with_input(bs)
analyzer= pcvl.algorithm.Analyzer(p,[bs],'*')
pcvl.pdisplay(analyzer)


#for i,j in sv.items():
    #print(i,j)
    #analyzer= pcvl.algorithm.Analyzer(p,[i],'*')
    #pcvl.pdisplay(analyzer)
parameters1 = np.random.normal(size=(2*(m-1)**2)-1) 
parameters2 = np.random.normal(size=(2*(m-1)**2)) 
px = 0
px1 = 0
#simulator_backend = pcvl.BackendFactory().get_backend("SLOS")
#print(len(parameters1))
for state,prob in zip(analyzer.output_states_list,analyzer.distribution[0]):
    
    new_state = state[1:]
    ms = state[0]
    #parameters1 = np.append(parameters1,ms)
    #print(parameters1)
    u1 = pcvl.Unitary(pcvl.Matrix.random_unitary(m-1,  np.append(parameters1,ms)), name = "U1")
    bs = pcvl.Unitary(pcvl.Matrix.random_unitary(m-1,  parameters2), name = "BS")
    p1=pcvl.Processor("SLOS",u1)
    p1.with_input(new_state)
    analyzer1= pcvl.algorithm.Analyzer(p1,[new_state],'*')
    print(prob)
    pcvl.pdisplay(analyzer1)
    px+= prob
    #px1 = 0
    #for state1,prob1 in zip(analyzer1.output_states_list,analyzer1.distribution[0]):
        #print(prob1)
        #px1+=prob*prob1
        #bs = pcvl.Unitary(pcvl.Matrix.random_unitary(m-1,  parameters2), name = "BS")
        
    
    
        #print(state1,prob1)
    
print(px)
#print(px1)
#parameters1 = np.random.normal(size=(2*(m-1)**2)-1)

#u1 = pcvl.Unitary(pcvl.Matrix.random_unitary(m-1, parameters1), name = "U1")
#len(parameters1)
#p=pcvl.Processor("SLOS",u1)
#p.with_input(bs)
#analyzer= pcvl.algorithm.Analyzer(p,[bs],'*')
#pcvl.pdisplay(analyzer)


In [ ]:
analyzer.output_states_list
analyzer.distribution[0]


r = 0*pcvl.StateVector(m*[0])

for state,prob in zip(analyzer.output_states_list,analyzer.distribution[0]):
    #print(state,prob)
    r += np.sqrt(prob)*pcvl.StateVector(state)
#r = pcvl.StateVector(0*"|0,0,0,0>")

# += pcvl.StateVector("|0,1,1,1>")
print(r)
#map_measure_sv = r.measure(0)
#for s, (p, sv) in map_measure_sv.items():
#    print(s, p, sv)
r.measure(0)